In [ ]:
%ShowTypes on

val sqlContext = new org.apache.spark.sql.SQLContext(sc)

// For implicit transformation of RDDs to DataFrames
import sqlContext.implicits._

// For telling Spark to look in the local file system
import java.io._
def localpath(path: String): String = {
    "file://" + new java.io.File(".").getCanonicalPath + "/" + path
}

// For timing expression evaluation
def time[R](block: => R): R = {
    val start: Long = System.nanoTime()
    val result = block
    val end: Long = System.nanoTime()
    val duration: Double = (end - start) / 1000000000.0
    println("Elapsed time: " + duration + "s")
    result
}

println("Using Spark version " + sc.version)

# Scala Primer

[Scala](https://en.wikipedia.org/wiki/Scala_(programming_language) is JVM based and has both object-oriented and functional aspects.  Whereas Python is an older imperative language that has tried to graft in functional programming elements, Scala was built ground-up as a [functional language](https://en.wikipedia.org/wiki/Functional_programming), with appropriate concessions for [object-oriented programming](https://en.wikipedia.org/wiki/Object-oriented_programming).
- Scala is [strongly typed](https://en.wikipedia.org/wiki/Strong_and_weak_typing), and strongly-typed functional programming languages have the nice (empirical) property that if you can get the code to compile, it will almost always work as intended.
- Unlike many other typed programming languages, you do not have to annotate every variable because the compiler has a strong [type inference](http://www.scala-lang.org/old/node/127).
- Scala is JVM so it can directly tap into existing [Java libraries](http://docs.scala-lang.org/tutorials/scala-for-java-programmers.html), and has a thriving ecosystem of its own.
- Since Scala is JVM based, you *can* use it as a "better Java" and simply write Java-like code.  Don't do this.
- The downside of Scala is that the compiler is [notoriously slow](http://www.quora.com/Why-is-the-Scala-compiler-so-slow).

So, why bother to learn it?
- Knowing more languages gives you more options (except for horrible things like [Whitespace](https://en.wikipedia.org/wiki/Whitespace_(programming_language))
- It's explicitly meant to be functional, so it translates well to distributed computing (where mutability gets confusing)
- Spark is written in it (we'll cover Spark later, it's a much more modern distributed computing framework)
- It's compiled, which comes with a slew of benefits
- It has a REPL for testing (like Python)


## Basic Syntax

Not space sensitive (other than newlines), so blocks are demarcated with curly brackets `{}` or parenthesis `()` depending on context.  Comments are C-style, `//` for single line commments and `/* */` for multi-line

We'll cover making classes in a bit

In [ ]:
//Creating a value
val x = 3*4

val li = List(1,2,3,4,5)

/* Functions are similar to python, but need typing in the definitions.  Return types
   can be omitted, but it's generally a good idea to include them.  Note no "return" statement,
   it returns the value of the last line*/ 
def cube(x: Int): Int = {
  //I do nothing
  x*x*x
}

//Method calls on classes work as in java/C++/etc
val howdy = "Howdy"
println("Howdy length = " + howdy.length)

//for loops work basically the same, in their most primitive form
//This loop would be considered extremely bad form in Scala
for(x <- li) {
  //The x value here will automatically get converted to a string,
  //Scala knows + on strings involves turning things in to strings
  println("a = " + x)
}

//while loops exist
var i = 0
while(i < 7) {
    println(s"i = $i")
    i += 1
}

//But they're not used as much as you'd think, since all collections have map, reduce, 
//and filter and most have flatMap.  These all exist in python as well
//But no list comprehensions.  But you can play for loop games

//lambdas are written (input vars) => computation
println(li.reduce((x,y) => x + y))
//underscore is lambda shorthand, you'll unfortunately see it *a lot*
println(li.map(cube(_)))
//And in special cases you can omit it and let the compiler figure it out
println(li.map(cube).reduce(_ + _))
println(li.filter(_ % 2 == 1))
//flatMap is very important in functional programing (hello Monads!)
def makeCopies(n: Int): List[Int] = {
    List.fill(n)(n)
}
println(li.flatMap(makeCopies))

//Tuples also exist, and work about the same as Python, but they're typed
val tup = ("bob",7)
//and we get their elements with ._1 and ._2 (don't ask)
println(tup._1)
//And we can tear them apart
val (x,y) = tup
println(s"$x, $y came from $tup")

**Questions:**

We'll be seeing lambdas A LOT in Scala.  Let's create a few.
1. Concatenate all the strings in a list
2. Get the lengths of those strings in the list
3. Starting from either point, find the total number of characters

## Values vs. Variables

Values or variables are defined with `val` or `var` respectively, afterwards variables can be set with `=`.  Immutable `val`s can only be set at creation

In [ ]:
var x = 1
x = 2   // but don't use vars!
val y = 1  // immutable - reassignment throws error
// y = 2
println("Functional!")
//Semi-colons *technically* end lines, but they're inferred if not there
print("I've got a semi-colon, so ... "); println("I'm a different line/expression")

In [ ]:
val x: Int = 1+1         // you can annotate the type
val y = 1+1              // or not, the compiler will infer it (usually)
assert(x == y)
println(s"x: $x, y: $y") // string interpolation

### Everything is a value
All statements return *something*.  If that thing is meant to be an action, it will (hopefully) return *Unit* to indicate that.  This is very much in line with strong typing - if you get a *Unit*, often written as `()`, you know a *side effect* of some nature occurred.

In [ ]:
val ret = println(s"I'm printing 7 * 3 = ${7*3}")
println(s"printing gives back the value ${ret}")

In [ ]:
val conditional = if(7 > 3) {
  "Yep"
} else {
  "No"
}
println(conditional)
println(conditional.getClass)

Functional programming treats computation as the evaluation of mathematical functions and avoids changing state and mutable data. That way, every function call with the same inputs yields the same outputs.  *This is the absolute key concept in functional programming.* Functions *do not* have internal state that changes.  In Scala, this is extended to objects as well, so generally "changing" an object doesn't change it, but rather returns a new instance that is an altered copy of the old one.

These cells are also expressions. Deleting the print statement in the above cell will cause the interpreter to attempt parsing the commented-out line: the last line of code in the block being what the expression evaluates to.

**Questions:**

1. What would be something with a side effect?
2. Something without?
3. Why does println return `()`?  What other options would make sense?

### Almost everything is an object
Inheriting this from Java, then running with it, this includes base types (Int, Float, String, etc), lambdas, nearly everything that isn't a control structure.

In [ ]:
// + is actually a method of the "Int" class
assert(x.+(y) == x + y)

//Lambda syntax, called "anonymous functions" in Scala
//Can be defined with underscores as well
//The type annotation is usually not needed if the lambda is used in a context
val square: Int => Int = x => x * x

println(square.getClass)

// you can also omit traditional class method invocation syntax (but probably shouldn't)
// this is how + works
(1 until 5) map square foreach println
//Unrolls to
((1.until(5)).map(x => x*x)).foreach(println(_))

### Basic types
`Int`, `String`, `Float`, `Double`, `Char` do exactly what you'd expect

## Basic data structures

In [ ]:
//List is the most commonly used, implemented as a linked list
val smallList = List(1, 2, 3)
val smallMap = Map('a' -> 1, 'b' -> 2)        // equiv: Map(('a', 1), ('b', 2))
val smallSet = Set('a', 'b')
val smallArray = Array(1, 2, 3)
val optSome = Some(2)                         // Option type
val optNone = None

val list = (0 until 10).toList                 // alt: (0 to 9).toList
val largerNumbers = list.map(_ + 1000)         // alt: list.map(x => x + 1000)
val evens = list.filter(_ % 2 == 0)
val set = list.toSet                           // alt: list.toSet(), parens optional
val alphabet = ('a' to 'z').toList
val alphabetMap = alphabet.zipWithIndex.toMap  // like python's enumerate function

// foreach is like map but when you don't expect a return value
smallList.foreach(println)

## List vs. Array

A Scala `List` is implemented as a linked list.  An `Array` is really a fixed length in memory buffer (just like the Java array).  They both inherit from the more generic `Seq` interface.

In [ ]:
// construct list
val list = 1 :: 2 :: 3 :: Nil // 1 -> 2 -> 3 -> end 

// and use it
def sum(list: List[Int]): Int = list match {
    case head :: tail => head + sum(tail)
    case Nil => 0
}
assert(sum(list) == 6)

// construct an array
val array = new Array[Int](3)
array(0) = 1
array(1) = 2
array(2) = 3
// array(4) = 4  // this throws an exception

assert(list(2) == array(2))  // select
assert((0 :: list) == (0 +: array).toList)  // prepend
assert((list :+ 4) == (array :+ 4).toList)  // append

**Question:**
- What is the running time of `list(n)` in Scala (and Python)?
- What is the running time of `array(n)`?
- What is the cost of prepending to `list` (i.e. `a :: list`)?
- What is the cost of prepending to `array` (i.e. `a +: list`)?
- What is the cost of appending to `list` (i.e. `list :+ a`)?
- What is the cost of appending to `array` (i.e. `list :+ a`)?

## Chaining and anonymous functions

These may seem simple, and they are, but they're probably the most important concept if you plan on using Spark

In [ ]:
val urls = List(
    "http://www.google.com",
    "http://www.google.com?q=scala",
    "http://www.yahoo.com",
    "http://www.bing.com"
)

// chained maps
// note that due to the underlying REPL we need to enclose chained methods in braces
// This isn't needed in compiled code
val names1 = ( urls
    .map(_.split("//").last)
    .map(_.split('.')(1))
)

// multiline anonymous function
val names2 = urls.map({x =>
    val domain = x.split("//").last
    domain.split('.')(1)
})

assert(names1 == names2)

## Making your own classes

In [ ]:
class Complex(real: Double, imaginary: Double) {
   //Class variables, you'll be able to access these from outside
   val re = real
   val im = imaginary
   
   def +(other: Complex): Complex = {
     new Complex(re + other.re, im + other.im)
   }
   
   //Can omit return type, not a good idea
   def *(other: Complex) = {
     //Fill me in
   }
   
   //The toString class controls how things are printed
   //to the screen
   override def toString(): String = {
     s"$re + $im i"
   }
}

val c = new Complex(7.0,3.0)
val d = new Complex(1.1, 2.2)

println(c.im)

//you get the operator notation for free!
println(c + d)

**Questions:**

1. As written, what is the return type of \*?  What should it be?
2. Let's fill it in, what is the return type now?

## Equality in Scala

*It depends on what the meaning of the word 'is' is*: [a famous ex-president.]

There are [two notions](https://en.wikipedia.org/wiki/Relational_operator#Object_identity_vs._content_equality) of `==`:
- *Object identity* or *shallow equality* is when two objects are pointing  at the same thing (in C, we might call this a pointer check)
- *Structural equality* or *deep equality* is when the two objects reference equivalent sturctures

This can be a little confusing because `==` can mean both:

In [ ]:
val a1 = (0 to 10).toArray
val a2 = (0 to 10).toArray
assert(a1 != a2)  // legacy java notion of comparison

val l1 = (0 to 10).toList
val l2 = (0 to 10).toList
assert(l1 == l2)

## Monads and map, flatMap, filter, foreach

It is Pythonic to operate on lists -- element-wise operations, filtering, etc. -- by using list comprehensions.  In many other languages, starting with Lisp but extending to many "functional" programming languages, a different style is preferred:

The idea is that if `f` is a function, then one thinks of the application
>          
    list   |---->   [ f(x) for x in list ]

on lists as a function of _two_ arguments: `f` and `list`.  The idea of viewing the function `f` as a parameter is typical in functional programming languages, and can be taken as a definition of the later term.

Some common idioms in this style, with Pythonic equivalents, are:

- Apply `f` element-wise to `list`:
    ``` python
    map(f, list) == [ f(x) for x in list ]
    ```
- Filter `list` using `f`:
    ``` python
    filter(f, list) == [ x for x in list if f(x) ]
    ```
- Here `f` is a function that eats elements (of the type contained in `list`) and spits out lists, and `flatMap` first applies `f` element-wise to the elements of `list` and then _flattens_ or _concatenates_ the resulting lists.  It is sometimes also called `concatMap`.
    ``` python
    flatMap(f, list) == [ x for y in list for x in f(y) ]
    ```
- `reduce(f, list[, initial])`: Here `f` is a function of _two_ variables, and folds over the list applying `f` to the "accumulator" and the next value in the list.  That is, it performs the following recursion
    $$    a_{-1} = \mathrm{initial} $$
    $$    a_i = f(a_{i-1}, \mathrm{list}_i) $$
    with the final answer being $a_{\mathrm{len}(\mathrm{list})-1}$.  (If initial is omitted, just start with $a_0 = \mathrm{list}_0$.)  For instance,
    ``` python           
    reduce(lambda x,y: x+y, [1,2,3,4]) == ((1+2)+3)+4 == 10
    ```

*Note*: This is where the name "mapReduce" comes from!

### Examples:
Below are some examples of iterations.

In [ ]:
// All factors of a number: what type does this return?
def factors(num: Int) = {
    (1 to num).filter(num % _ == 0).toList  // last line in function is returned
}

val factors6 = factors(6)

// count of all factors up to n
def factorSum(num: Int) = {
    val allFactors = (1 to num).flatMap(factors)
    allFactors.groupBy(x => x).map(t => (t._1, t._2.length))  // can map over a HashMap
}

val factorSum10 = factorSum(10)

In [ ]:
// largest palindrome that is the product of two 3-digit numbers

// this is hard to read
val r1 = ((100 to 999).view
            .flatMap(i => (i to 999).map(i * _))
            .filter(n => n.toString == n.toString.reverse)
            .max)

// for comprehension is easier to read
val r2 = (for {
    i <- 100 to 999
    j <- i to 999
    val n = (i * j)
    val s = n.toString
    if s == s.reverse
} yield { n }).max

assert(r1 == r2)

## Options and map

`Option` is a funny collection because it contains at most a single object.  But it turns out to be the most useful and very powerful when combined with `map` and `flatMap`.  The type system checks that you've considered all the corner cases.

In [ ]:
// so what are options for?
import java.util.Arrays

// binary search does not always return an index, so the return type should be optional
def binarySearch(array: Array[Char], key: Char): Option[Int] = {
    Arrays.binarySearch(array, key) match {
        case index if index < 0 => None
        case index => Some(index)
    }
}

val targetArray = ('a' to 'z').toArray

assert(binarySearch(targetArray, 'b') == Some(1))
assert(binarySearch(targetArray, 'B') == None)

// character after 'b' (don't forget the null case)
def letterAfter1(array: Array[Char], key: Char): Option[Char] = {
    binarySearch(array, key) match {
        case Some(index) => if (array.isDefinedAt(index + 1)) {
            Some(array(index + 1))
        } else {
            None
        }
        case None => None
    }
}

// an even more idiomatic implementation
def letterAfter2(array: Array[Char], key: Char): Option[Char] = {
    binarySearch(targetArray, key).
        flatMap(index => targetArray.drop(index+1).headOption)
}

assert(letterAfter1(targetArray, 'b') == letterAfter2(targetArray, 'b'))
assert(letterAfter1(targetArray, 'z') == letterAfter2(targetArray, 'z'))
assert(letterAfter1(targetArray, 'B') == letterAfter2(targetArray, 'B'))

## Classes and objects

### A simple example

In [ ]:
// Companions must be defined together so we'll simulate :paste mode with braces.
{
    trait PointTrait {
        def square(x: Double) = x * x // don't even need to declare return type
    }

    class Point(val x: Double, val y: Double) extends PointTrait {
        // val means constructor arguements are public    
        def distance(p: Point): Double = {
            math.sqrt(square(p.x - x) + square(p.y - y))
        }
    
        val norm = math.sqrt(square(x) + square(y))  // as if in constructor (e.g. __init__)
    }

    object Point extends PointTrait

assert(new Point(0.0, 0.0).distance(new Point(1.0, 1.0)) == new Point(1.0, 1.0).norm)
}

### A more complex example

In [ ]:
class Stack[T] {  // T is a type parameter (see usage)
    var data: List[T] = Nil
    
    def push(x: T) {  // no return value => don't need '='
        data = x :: data
    }

    def pop: Option[T] = data match {  // matches on type of 
        case init :: tail => {  // matches non-empty list
            data = tail
            Some(init)
        }
        case Nil => None  // matches empty list
    }
}

val stack = new Stack[Int]
stack.push(2)
stack.push(3)
assert(stack.pop == Some(3))
assert(stack.pop == Some(2))
assert(stack.pop == None)

A good [reference](https://twitter.github.io/scala_school/type-basics.html) for Scala typing.

## Pass arguments by name or value

In [ ]:
def passValue(x: Int) = {
  println("x1=" + x)
  println("x2=" + x)
}

def passAnonymousFunction(x: => Int) = {
  println("x1=" + x)
  println("x2=" + x)
}

def printfn() = {
    println("hello")
    3
}

passValue(printfn)
println("-----")
passAnonymousFunction(printfn)

Remember, everything is an expression.

`printfn` evaluates to the integer 3. But it's also a function that evaluates to the integer 3.

If we pass it in to another function as a value, it will be evaluated and the value passed in used when asked for.

If we pass it in to another function as a function, it will not be evaluated until asked for, and if you ask again, it must be evaluated again.

*Question*: Can you think of some benefits of this kind of "lazy evaluation"? What about the drawbacks?

## Try, catch, and finally
Try and catch in Scala is a lot like other languages.

In [ ]:
try {
    println("Before the exception")
    throw new IllegalArgumentException ("")
    println("After the exception")
} catch {
    case e: Exception => println("got an exception: " + e)
} finally {
    println("print me no matter what")
}

**Exercise**: Exceptions have a method `printStackTrace` which requires no arguments. Modify the above code to print not just the name of the error, but the entire stacktrace.

## Advantages and disadvantages of Scala?
1. It reduces performance overhead
2. Access to the latest and greatest
3. Understand the underlying philosophy of computation that Spark inherits from being developed in Scala
4. One language to rule them all:
> With Spark and Scala, the experience is different, because you’re using the same language for everything. You’re writing Scala to retrieve data from the cluster via Spark. You’re writing Scala to manipulate that data locally on your own machine. And then — and this is the really neat part — you can send Scala code into the cluster so that you can perform the exact same transformations that you performed locally on data that is still stored in the cluster. It’s difficult to express how transformative it is to do all of your data munging and analysis in a single environment, regardless of where the data itself is stored and processed. It’s the sort of thing that you have to experience for yourself to understand, and we wanted to be sure that our recipes captured some of that same magic feeling that we felt when we first started using Spark.

## Additional resources

- [Scala School](https://twitter.github.io/scala_school/)
- [Scala for Python programmers](http://bugra.github.io/work/notes/2014-10-18/scala-basics-for-python-developers/)
- [The Official Scala-Lang Learning Page](http://www.scala-lang.org/documentation/)
- [Another tutorial on Scala](http://www.scala-lang.org/docu/files/ScalaTutorial.pdf)
- [Scala For The Impatient](http://fileadmin.cs.lth.se/scala/scala-impatient.pdf)

#### Exit Tickets
1. Explain flatMap to a layman.
1. Write a fully typed function that accepts an integer as input, returns the integer if it is divisible by k, and if not returns None.

*Copyright &copy; 2015 The Data Incubator.  All rights reserved.*